In [ ]:

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c plant-seedlings-classification

In [ ]:
!unzip '/content/plant-seedlings-classification.zip'


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm

In [ ]:
train_dir = '/content/train'


In [ ]:
img_width, img_height = 150, 150
batch_size = 32


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(12, activation='softmax')
])


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10
)


model.save('image_classification_model.h5')

Increasing epoch will improve accuracy,but since it takes a lot of time,I reduced the number of epoch and thus accuracy.

In [ ]:
unseen_dir = '/content/test'
class_labels=['Black-grass','Charlock','Cleavers','Common Chickweed','Common wheat','Fat Hen','Loose Silky-bent','Maize','Scentless Mayweed','Shepherds Purse','Small-flowered Cranesbill','Sugar beet']


file_names = os.listdir(unseen_dir)


predictions_df = pd.DataFrame(columns=['file', 'species'])

for file_name in tqdm(file_names, desc="Making predictions"):

    img_path = os.path.join(unseen_dir, file_name)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = img_array.reshape((1, img_width, img_height, 3))
    img_array = img_array / 255.0


    prediction = model.predict(img_array)
    predicted_class_index = prediction.argmax()


    predicted_class = class_labels[predicted_class_index]


    predictions_df = pd.concat([predictions_df, pd.DataFrame({'file': [file_name], 'species': [predicted_class]})], ignore_index=True)


predictions_df.to_csv('submission.csv', index=False)